# Analysis of Reversi Data

In [ ]:
import struct
import glob
from typing import List, Tuple, Dict

In [ ]:
NUM_OF_READ = 20
CUT_THRESHOLD = 50

In [ ]:
def load_wtb_file(name: str) -> List[Tuple[int, List[Tuple[int]]]]:
    with open(name, "rb") as file:
        headers = file.read(16)
        (year1, year2, month, day, record_num, _, year, size, record_type, depth, _) = struct.unpack("BBBBIHHBBBB", headers)

        record = []
        for _ in range(record_num):
            (contest_id, black_id, white_id, black_num, black_best) = struct.unpack("HHHBB", file.read(8))
            data = []
            for _ in range(60):
                data.append(struct.unpack("B", file.read(1))[0])
            record.append((black_best, data))
        return record

In [ ]:
data = []
for file in glob.glob("./data/*.wtb"):
    data += load_wtb_file(file)

In [ ]:
def concat_moves(moves: List[int]) -> str:
    return ",".join(map(str, moves))

def split_moves(moves: str) -> List[int]:
    return list(map(int, moves.split(",")))

In [ ]:
assert split_moves(concat_moves([1, 2, 3])) == [1, 2, 3]

In [ ]:
Data = Dict[Tuple[int, str], Dict[int, Tuple[int, float]]]

In [ ]:
move_lists: Data = { }
for i in range(1, NUM_OF_READ):
    for (best, move) in data:
        key = i, concat_moves(move[0:i])
        if key not in move_lists:
            move_lists[key] = { }
        if move[i] in move_lists[key]:
            (count, score) = move_lists[key][move[i]]
            move_lists[key][move[i]] = (count + 1, score + best)
        else:
            move_lists[key][move[i]] = (1, best)

In [ ]:
move_lists

In [ ]:
for num, move in move_lists.keys():
    for m in move_lists[num, move].keys():
        (count, score) = move_lists[num, move][m]
        move_lists[num, move][m] = (count, score / count)

In [ ]:
move_lists

In [ ]:
move_table = {}
for num, move in move_lists.keys():
    is_black = num % 2 == 0
    best_move = 0
    best_score = -1000 if is_black else 1000
    appeard = 0

    for m in move_lists[num, move].keys():
        (count, score) = move_lists[num, move][m]
        appeard += count
        if is_black:
            if score  > best_score:
                best_score = score
                best_move = m
        else:
            if score < best_score:
                best_score = score
                best_move = m

    if appeard >= CUT_THRESHOLD:
        move_table[num, move] = best_move

In [ ]:
move_table

In the original data, the first move is fixed to `56`. We are going to flip boards to make it easy to search a best move.

In [ ]:
move_table_complete = { }
for num, move in move_table.keys():
    moves = split_moves(move)
    best_move = move_table[(num, move)]

    move_table_complete[(num, move)] = best_move

    def flip1(x: int) -> int:
        return (x % 10) * 10 + (x // 10)
    def flip2(x: int) -> int:
        return (9 - x // 10) * 10 + (9 - x % 10)
    def flip3(x: int) -> int:
        return flip2(flip1(x))

    move_table_complete[(num, concat_moves(list(map(flip1, moves))))] = flip1(best_move)
    move_table_complete[(num, concat_moves(list(map(flip2, moves))))] = flip2(best_move)
    move_table_complete[(num, concat_moves(list(map(flip3, moves))))] = flip3(best_move)

In [ ]:
move_table_complete

In [ ]:
with open("preprocessed.txt", "w") as file:
    for key in move_table_complete.keys():
        for move in split_moves(key[1]):
            pos = chr(ord('A') + move // 10 - 1) + str(move % 10)
            file.write(f"{pos}")
        move = move_table_complete[key]
        pos = chr(ord('A') + move // 10 - 1) + str(move % 10)
        file.write(f" {pos}\n")